In [1]:
import pandas
import datetime
import numpy
from aacgmv2 import convert_mlt, convert
from pyigrf12 import gridigrf12

In [16]:
# Some constants
cutOffMLTMLATUniqCnt = 4
delLatDist = 111. * 1e3 # 1 deg lat is 111 km

55500.0


In [3]:
velsDataDF = pandas.read_csv( "../data/processed-vels-geomag-fin.txt", sep=' ',\
                            infer_datetime_format=True,\
                            parse_dates=["date"])
# Get actual SAPS azimuths
velsDataDF["azim"] = -90.-velsDataDF["azim"]
velsDataDF.head()
# Filter out some values
# When there aren't good number of measurements at lats
# discard those values
cntMLTPntsDF = velsDataDF[ ["dst_bin", "normMLT", "MLAT"] \
                    ].groupby( ["dst_bin", "normMLT"] \
                    )["MLAT"].nunique().reset_index()
cntMLTPntsDF.columns = ["dst_bin", "normMLT", "MLATCntUnq"]
cntMLTPntsDF = cntMLTPntsDF[ cntMLTPntsDF["MLATCntUnq"] >=\
                    cutOffMLTMLATUniqCnt ][ ["dst_bin", "normMLT"]\
                        ].reset_index(drop=True)
velsDataDF = pandas.merge( velsDataDF, cntMLTPntsDF,\
                    on=["dst_bin", "normMLT"], how="inner" )
velsDataDF["hour"] = velsDataDF["date"].apply(lambda x: x.strftime('%H'))
velsDataDF["minute"] = velsDataDF["date"].apply(lambda x: x.strftime('%M'))
velsDataDF["dtStr"] = velsDataDF["date"].apply(lambda x: x.strftime('%Y%m%d'))

In [4]:
def get_geo_coords(row):
    if row["normMLT"] < 0: 
        currMLT = row["normMLT"] + 24.
    else:
        currMLT = row["normMLT"]
    currMlon = convert_mlt( currMLT, row["date"] , m2a=True )
    glat, glon = convert( row["MLAT"], currMlon , 300, row["date"], a2g=True)
    return str(glat) + "__" + str(glon)

velsDataDF["gcoord"] = velsDataDF.apply( get_geo_coords, axis=1 )
velsDataDF["glat"] = [ float( x.split("__")[0] ) for x in velsDataDF["gcoord"] ]
velsDataDF["glon"] = [ float( x.split("__")[1] ) for x in velsDataDF["gcoord"] ]

In [5]:
def get_efield(row):
    Bx,By,Bz,Bf, yeardec = gridigrf12( row["date"], 0, 1, 300., row["glat"], row["glon"] )
    Vy = row["vSaps"] * numpy.round( numpy.sin(numpy.deg2rad( row["azim"] )), 2 )
    Vx = row["vSaps"] * numpy.round( numpy.cos(numpy.deg2rad( row["azim"] )), 2 )
    Vz = 0.
    Ex = Bz[0] * -1 * Vy * 1e-9 # nT
    Ey = Bz[0] * Vx * 1e-9 # nT
    Ez = Bx[0] * Vy * 1e-9 - By[0] * Vx * 1e-9 # nT
    # Round electric fields by 2
    Ex = numpy.round( Ex * 1e3 , 4 ) # mV/m
    Ey = numpy.round( Ey * 1e3 , 4 ) # mV/m
    Ez = numpy.round( Ez * 1e3 , 4 ) # mV/m
    return str(Ex) + "__" + str(Ey) + "__" + str(Ez)

velsDataDF["Efield"] = velsDataDF.apply( get_efield, axis=1 )
velsDataDF["Ex"] = [ float( x.split("__")[0] ) for x in velsDataDF["Efield"] ]
velsDataDF["Ey"] = [ float( x.split("__")[1] ) for x in velsDataDF["Efield"] ]
velsDataDF["Ez"] = [ float( x.split("__")[2] ) for x in velsDataDF["Efield"] ]
velsDataDF["Efield"] = numpy.sqrt( numpy.square(velsDataDF["Ex"]) +\
                                 numpy.square(velsDataDF["Ey"]) +\
                                 numpy.square(velsDataDF["Ez"]))

In [6]:
# Asy Inds
asyDF = pandas.read_csv( "../data/Asy_processed.txt", sep=' ' )
asyDF["date"] = pandas.to_datetime(asyDF["datetimeStr"], format='%Y%m%d-%H-%M')
asyDF["hour"] = asyDF["date"].apply(lambda x: x.strftime('%H'))
asyDF["minute"] = asyDF["date"].apply(lambda x: x.strftime('%M'))
asyDF["dtStr"] = asyDF["date"].apply(lambda x: x.strftime('%Y%m%d'))
asyDF.head()

,datetimeStr,AsyD,AsyH,SymD,SymH,date,hour,minute,dtStr
0,20110101-00-00,17.0,13.0,1.0,-10.0,2011-01-01 00:00:00,00,00,20110101
1,20110101-00-01,17.0,13.0,1.0,-10.0,2011-01-01 00:01:00,00,01,20110101
2,20110101-00-02,17.0,13.0,1.0,-10.0,2011-01-01 00:02:00,00,02,20110101
3,20110101-00-03,17.0,13.0,1.0,-10.0,2011-01-01 00:03:00,00,03,20110101
4,20110101-00-04,17.0,13.0,1.0,-10.0,2011-01-01 00:04:00,00,04,20110101


In [7]:
velsDataDF = pandas.merge( velsDataDF, asyDF,\
                         on=["dtStr", "hour", "minute"], how='inner')
velsDataDF = velsDataDF [ ['azim','azimStd', 'delMLT', 'goodFit',\
                      'MLAT', 'normMLT', 'vSaps', 'velSTD',\
                      'endPtMLAT', 'endPtNormMLT', 'date_x','dtStr',\
                      'hour', 'minute', 'dst_date', 'dst_index', 'dst_bin',\
                      'datetimeStr_x', 'AE', 'AL', 'AO', 'AU', 'count', 'maxCount',\
                       'probOcc', 'AsyD', 'AsyH', 'SymD', 'SymH',\
                        'glat', 'glon', 'Efield', 'Ex', 'Ey', 'Ez', "potential"] ]
velsDataDF.columns = ['azim','azimStd', 'delMLT', 'goodFit',\
                      'MLAT', 'normMLT', 'vSaps', 'velSTD',\
                      'endPtMLAT', 'endPtNormMLT', 'date','dtStr',\
                      'hour', 'minute', 'dst_date', 'dst_index', 'dst_bin',\
                      'datetimeStr', 'AE', 'AL', 'AO', 'AU', 'count', 'maxCount',\
                       'probOcc', 'AsyD', 'AsyH', 'SymD', 'SymH',\
                        'glat', 'glon', 'Efield', 'Ex', 'Ey', 'Ez']

In [8]:
# Divide asymH index into bins
asyHBins = [ 0, 30, 60, 90, 180 ]
velsDataDF = pandas.concat( [ velsDataDF, \
                    pandas.cut( velsDataDF["AsyH"], \
                               bins=asyHBins ) ], axis=1 )
velsDataDF.columns = ['azim','azimStd', 'delMLT', 'goodFit',\
                      'MLAT', 'normMLT', 'vSaps', 'velSTD',\
                      'endPtMLAT', 'endPtNormMLT', 'date','dtStr',\
                      'hour', 'minute', 'dst_date', 'dst_index', 'dst_bin',\
                      'datetimeStr', 'AE', 'AL', 'AO', 'AU', 'count', 'maxCount',\
                       'probOcc', 'AsyD', 'AsyH', 'SymD', 'SymH',\
                        'glat', 'glon', 'Efield', 'Ex', 'Ey', 'Ez', 'AsyH_bin']
velsDataDF.head()

,azim,azimStd,delMLT,goodFit,MLAT,normMLT,vSaps,velSTD,endPtMLAT,endPtNormMLT,...,SymD,SymH,glat,glon,Efield,Ex,Ey,Ez,potential,AsyH_bin
0,-79.412573,1.794202,0.5,True,62.0,-6.0,306.738497,11.198685,62.06,-6.30,...,2.0,-48.0,52.550383,-100.904280,15.307472,14.6255,2.6863,-3.6328,849.564686,"(30, 60]"
1,-78.075135,1.005434,0.5,True,61.5,-6.0,296.893207,6.211472,61.56,-6.29,...,2.0,-48.0,52.025938,-100.963274,14.864103,14.0997,3.0214,-3.6071,824.957736,"(30, 60]"
2,-82.467569,3.532155,0.5,True,60.0,-6.0,220.239731,11.464162,60.03,-6.22,...,2.0,-48.0,50.455151,-101.132638,10.899579,10.4296,1.3695,-2.8546,604.926648,"(30, 60]"
3,-84.987310,1.491959,0.5,True,60.5,-6.0,233.815511,5.963564,60.52,-6.23,...,2.0,-48.0,50.978340,-101.077372,11.668698,11.2349,1.0111,-2.9855,647.612735,"(30, 60]"
4,-79.086184,0.914397,0.5,True,62.5,-6.0,343.746070,6.854657,62.57,-6.34,...,2.0,-48.0,53.075283,-100.843875,17.226031,16.4528,3.1898,-3.9833,956.044699,"(30, 60]"


In [19]:
mltVelsVarsDF = velsDataDF[ ["normMLT", "MLAT","AsyH_bin", "vSaps", "Efield"] \
                    ].groupby( ["normMLT", "MLAT", "AsyH_bin"] ).mean().reset_index()
mltVelsVarsDF.columns = ["normMLT", "MLAT", "AsyH_bin", "mean_vel", "mean_efld"]
stdVelsVarsDF = velsDataDF[ ["normMLT", "MLAT", "AsyH_bin", "vSaps", "Efield"] \
                    ].groupby( ["normMLT", "MLAT", "AsyH_bin"] ).std().reset_index()
stdVelsVarsDF.columns = ["normMLT", "MLAT", "AsyH_bin", "std_vel", "std_efld"]
mltVelsVarsDF = pandas.merge( mltVelsVarsDF, stdVelsVarsDF, on=["normMLT", "MLAT", "AsyH_bin"] )
mltVelsVarsDF = mltVelsVarsDF.dropna().reset_index(drop=True)
mltVelsVarsDF.head()

,normMLT,MLAT,AsyH_bin,mean_vel,mean_efld,std_vel,std_efld
0,-7.0,56.5,"(30, 60]",916.695481,43.882040,180.465685,8.677802
1,-7.0,56.5,"(60, 90]",910.908751,43.436630,242.537748,11.510254
2,-7.0,56.5,"(90, 180]",955.350841,45.452521,220.046676,10.655408
3,-7.0,57.0,"(30, 60]",878.407048,42.256509,380.813290,18.331726
4,-7.0,57.0,"(60, 90]",910.114970,43.703457,229.581957,11.160234


In [1]:
# Arrays to store potential
potBinArr = []
potNrmMltArr = []
potMlatArr = []
potPotArr = []
# get uniq bins
for cbin in mltVelsVarsDF["AsyH_bin"].unique().tolist():
    # sel data only from the bin
    selLocsDF = mltVelsVarsDF[ mltVelsVarsDF["AsyH_bin"] == cbin ]
    for cmlt in selLocsDF["normMLT"].unique().tolist():
        # Get Uniq Lats at the MLT!
        selMltDF = selLocsDF[ selLocsDF["normMLT"] == cmlt ]
        # At each MLT we assume potential is zero at the lowest
        # latitude!
        prevPot = 0
        print cbin, cmlt
#         for cMlat in sorted( selMltDF["MLAT"].unique().tolist() ):
#             selLatsDF = selLocsDF[ (selLocsDF["MLAT"] == cMlat) ]
              # Remember we have 0.5 deg latitude res
              # so we take delLatDist/2
#             currPot = prevPot - selLatsDF[ selLatsDF["MLAT"] == cMlat ]["mean_efld"] * delLatDist/2. * 1e-3 # volts
#             potBinArr.append( cbin )
#             potNrmMltArr.append( potNrmMltArr )
#             potMlatArr.append( cMlat )
#             potPotArr.append( currPot )

# # Store the results in a DF
# potentialDF = pandas.DataFrame({
#             "AsyH_bin" : potBinArr,
#             "normMLT" : potNrmMltArr,
#             "MLAT" : potMlatArr,
#             "potential" : potPotArr
#             })
# print potentialDF

NameError: name 'mltVelsVarsDF' is not defined